In [1]:
import pandas
import numpy
from scipy import sparse
import networkx as nx
import matplotlib.pyplot as plt

# read the friends network data
friends_file = pandas.read_csv("gowalla_data/gowalla_friendship.csv")
friends_u = list(friends_file['userid1'])
friends_f = list(friends_file['userid2'])
friends_data = zip(friends_u,friends_f)

# user-item matrix generated in earlier file
user_item_sparse = sparse.load_npz('generated_data/user_item_matrix.npz')
user_item_csr = user_item_sparse.tocsr()

user_loc_f = numpy.load('generated_data/user_item_matrix_columns.npz')
user_list = list(user_loc_f['arr_0'])
loc_list = list(user_loc_f['arr_1'])

# create user_idx and loc_idx
user_idx = {}
idx = 0
for u in user_list:
    user_idx[u] = idx
    idx += 1

loc_idx = {}
idx = 0
for l in loc_list:
    loc_idx[l] = idx
    idx += 1
#

P_Q_f = numpy.load('generated_data/user_item_decomposed.npz')
P = P_Q_f['arr_0']
Q = P_Q_f['arr_1']

In [2]:
# store the data for webui in 3 parts
num1 = len(friends_u)//3
num2 = 2*len(friends_u)//3
u0 = friends_u[0:num1]
f0 = friends_f[0:num1]
u1 = friends_u[num1:num2]
f1 = friends_f[num1:num2]
u2 = friends_u[num2:]
f2 = friends_f[num2:]
numpy.savez('generated_data/friendship_data1.npz',u0,f0)
numpy.savez('generated_data/friendship_data2.npz',u1,f1)
numpy.savez('generated_data/friendship_data3.npz',u2,f2)

In [2]:
DG = nx.DiGraph()

# add nodes based on the user-list
user_set = set(user_list)
for u in user_list:
    DG.add_node(u)
    
# add edges: friends based on friend data
for e in friends_data:
    if e[0] in user_set and e[1] in user_set:
        DG.add_edge(e[0],e[1])

# friends network stat
print("Nodes = ",DG.number_of_nodes())
print("Edges = ",DG.number_of_edges())

Nodes =  145040
Edges =  1832987


In [3]:
def get_friends(userid,depth_limit=1):
    friends_e = list(nx.bfs_edges(DG,userid,depth_limit=depth_limit))
    friends = []
    for e in friends_e:
        friends.append(e[1])
    
    return friends

In [4]:
def has_friend_visited(friends,usern,locn):
    # we use this function to check if a friend 
    # of userid has visited this locid. 
    userid = user_list[usern]
    for friendid in friends:
        friendn = user_idx[friendid]
        if user_item_csr[friendn,locn] != 0:
            return True
    
    return False  

In [5]:
def recommend_locations_base(userid):
    usern = user_idx[userid]
    
    predictions = numpy.dot(P[usern,:][numpy.newaxis],Q.T) # we get the predictions by dot product of P.Q [userid]
    predictions = predictions[0] #drop the two-dimensions
    predictions_idx = [x for _,x in sorted(zip(predictions,range(len(predictions))),reverse=True)]

    recommendations = numpy.array(loc_list)[predictions_idx]
    return recommendations
       
locations = recommend_locations_base(3)
print(locations)

[420315  21714   9241 ...  17144  13791  13776]


In [6]:
def recommend_locations_without_friends(userid,K=10):
    recommendations = recommend_locations_base(userid)
    
    # filter the recommended locations based on:
    # 1. it should not be visited by userid

    filtered_not_visited = []
    n = 0
    usern = user_idx[userid]
    while len(filtered_not_visited) < K and n < len(recommendations):
        locn = loc_idx[recommendations[n]]
        # 1:
        if user_item_csr[usern,locn] == 0:
            filtered_not_visited.append(recommendations[n])

        n += 1

    return filtered_not_visited

In [7]:
def recommend_locations_with_friends(userid,K=10):
    recommendations = recommend_locations_base(userid)
    
    # filter the recommended locations based on:
    # 1. it should not be visited by userid
    # 2. it should be visited by one of his friends
    friends = get_friends(userid)
    filtered_not_visited = []
    filtered_not_visited_and_friend_visited = []
    n = 0
    usern = user_idx[userid]
    while len(filtered_not_visited_and_friend_visited) < K and n < len(recommendations):
        locn = loc_idx[recommendations[n]]
        # 1 and 2:
        if user_item_csr[usern,locn] == 0:
            filtered_not_visited.append(recommendations[n])
        
        if user_item_csr[usern,locn] == 0 and has_friend_visited(friends,usern,locn):
            filtered_not_visited_and_friend_visited.append(recommendations[n])

        n += 1
    
    # its possible that user has no friends, so use the base
    # recommender for these cases
    if len(filtered_not_visited_and_friend_visited) == 0:
        return filtered_not_visited
    else:
        return filtered_not_visited_and_friend_visited


In [8]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

userw=widgets.SelectionSlider(
    options=user_list,
    value=3,
    description='UserId:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
#display(w)

def printer(userid):
    print("Recommended Locations:")
    print("With Friend Influence:")
    print(recommend_locations_with_friends(userid))
    print("Without Friend Influence:")
    print(recommend_locations_without_friends(userid))
    print("User Friends:")
    print(get_friends(userid,depth_limit=1))
    
interact(printer,userid=userw)
print("")

interactive(children=(SelectionSlider(continuous_update=False, description='UserId:', index=2, options=(1, 2, …